In [1]:
# Import necessary libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from InstructorEmbedding import INSTRUCTOR
import numpy as np
from tqdm import tqdm


hot_words = ["be careful", "destroy", "stranger"]  

cv_valid_dev_csv = '/Users/allen/Documents/Wav2vec_Project/asr/common_voice/cv-valid-dev.csv'
cv_valid_dev_df = pd.read_csv(cv_valid_dev_csv)
# print (cv_valid_dev_df)
model = INSTRUCTOR("hkunlp/instructor-large")
embedding_hot_word=model.encode(hot_words)

# embedding_sentence=model.encode([cv_valid_dev_df['text'][0]])
# similarities = cosine_similarity(embedding_hot_word,embedding_sentence)
# print(similarities)

similarity_threshold=0.5
similarity_column=[]
score_column=[]

for sentence in tqdm(cv_valid_dev_df['text'],desc="Computing Similarity"):
    # print (sentence)
    embedding_sentence=model.encode([sentence])
    similarities = cosine_similarity(embedding_hot_word,embedding_sentence)
    average = np.mean([x[0] for x in similarities])
    score_column.append(average)
    if average > similarity_threshold:
        similarity_column.append(True)
    else:
        similarity_column.append(False)

cv_valid_dev_df['similarity'] = similarity_column
cv_valid_dev_df['score'] = score_column
print(cv_valid_dev_df)

/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load INSTRUCTOR_Transformer
max_seq_length  512


/Users/allen/Documents/Wav2vec_Project/.venv/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_s

                            filename  \
0     cv-valid-dev/sample-000000.mp3   
1     cv-valid-dev/sample-000001.mp3   
2     cv-valid-dev/sample-000002.mp3   
3     cv-valid-dev/sample-000003.mp3   
4     cv-valid-dev/sample-000004.mp3   
...                              ...   
4071  cv-valid-dev/sample-004071.mp3   
4072  cv-valid-dev/sample-004072.mp3   
4073  cv-valid-dev/sample-004073.mp3   
4074  cv-valid-dev/sample-004074.mp3   
4075  cv-valid-dev/sample-004075.mp3   

                                                   text  up_votes  down_votes  \
0     be careful with your prognostications said the...         1           0   
1     then why should they be surprised when they se...         2           0   
2     a young arab also loaded down with baggage ent...         2           0   
3     i thought that everything i owned would be des...         3           0   
4     he moved about invisible but everyone could he...         1           0   
...                              

In [2]:
## Save CSV
cv_valid_dev_df.to_csv('cv-valid-dev-similarity.csv', index=False)